# **Installation of Packages**

First install packages like numpy, scikit-learn, matplotlib

In [1]:
!pip3 install numpy 
!pip3 install SPARQLWrapper
!pip3 install scikit-learn
!pip3 install PIL
!pip3 install pandas
!pip3 install webcolors
!pip3 install matplotlib

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


# **Importation of packages**

We import the necessary packages

In [2]:
import os, sys, json
import urllib, shutil
import random, time, webcolors, numpy
import requests
import matplotlib.pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from sklearn.cluster import KMeans, MiniBatchKMeans
import pandas as pd


# **Load Dataset**

In [3]:
endpoint_url = "https://query.wikidata.org/sparql"
img_data = {}

# Requête pour récupérer les infos de l'entité donnée - Q144/Chien par défaut
def query(wd='Q144', limit=5):
    return f"""SELECT ?item ?itemLabel ?pic
                WHERE {{
                  ?item wdt:P31 wd:{wd}.
                  ?item wdt:P18 ?pic.
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
                }}
                LIMIT {limit}"""

def get_results(endpoint_url, query):
    user_agent = 'MaximeCornaton/0.0 (https://github.com/MaximeCornaton/CPE_Data_Mining; maxime.cornaton@cpe.fr)'
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    # Boucle de tentative en cas d'erreur 429 Too Many Requests
    for i in range(3):
        try:
            return sparql.query().convert()
        except HTTPError as e:
            if e.response.status_code == 429:
                retry_after = int(e.response.headers.get('Retry-After', '60'))
                print(f"Too Many Requests. Tentative dans {retry_after} secondes.")
                sleep(retry_after)
            else:
                raise e
    
    raise Exception("Nombre maximum de tentatives pour atteindre Wikidata atteint.")

# Pour créer le dossier des images s'il n'existe pas
def createFolder(name='images'):
    if name not in os.listdir():
        os.mkdir(name)
    else:
        print('Dossier '+name+' déjà existant !')

# Pour vérifier si un dossier est vide - évitons de retélécharger des images inutilement.
def isEmpty(name='images'):
    if len(os.listdir(name)) == 0:
        return True
    return False

# Pour ajouter des éléments à la bdd
def addImg(wd, limit):
    results = get_results(endpoint_url, query(wd, limit))
    for result in results['results']['bindings']:
        name_img = result['item']['value'].split("/")[-1] + "-" + wd # nom donné aux images téléchargées "numéro d'image - numéro de catégorie"
        
        #img_data[name_img] = {'image': result['pic']['value']}
        
        img_data[name_img] = {}
        downloadImage(result['pic']['value'], name_img)

# Ajoute les images de toutes les entités spécifiées
def addAllImg(wd, limit):
    for w in wd:
        addImg(w, limit)

# Pour télécharger l'image d'un élément de la bdd
def downloadImage(url, name_img='img0'):
    full_path = 'images/' + name_img + '.jpg'
    urllib.request.urlretrieve(url, full_path)

# Si les images sont déjà téléchargées on initialise le tableau
def initAlreadyDownload(namefolder='images'):
    for image in os.listdir(namefolder):
        #img_data[image.split(".")[0]] = {'image': ""}
        img_data[image.split(".")[0]] = {}


# Pour télécharger toutes les images - limit par query
def downloadAllImages(wd=['Q144'], limit=5):
    createFolder()
    if not isEmpty():
       

        print('Images déjà téléchargées !')
        initAlreadyDownload()
        return False
    addAllImg(wd, limit)
    return True 

#Supprimer une image du dossier 
def removeImage(image):
    if os.path.isfile('images/'+image):
        os.remove('images/'+image)
        img_data.pop(image.split(".")[0])
    else:
        shutil.rmtree('images/'+image)

bdd = ['Q144','Q14660','Q23442','Q28803','Q3305213', 'Q16521', 'Q146', 'Q5', 'Q4022'] #Chien, drapeau, ile, sandwich, peinture, taxon, chat, être humain, riviere
downloadAllImages(wd=bdd,limit=50)
print(img_data)


Dossier images déjà existant !
Images déjà téléchargées !
{'Q86516809-Q144': {}, 'Q97356971-Q144': {}, 'Q958201-Q144': {}, 'Q783490-Q14660': {}, 'Q87783199-Q144': {}, 'Q96176417-Q144': {}, 'Q96372991-Q144': {}, 'Q97280926-Q144': {}, 'Q697098-Q144': {}, 'Q805807-Q144': {}, 'Q73521760-Q144': {}}


# **Adding Meta to the Dataset**

## *1. Exif* 

In [4]:
#Obtenir les metas d'une image
def getMetaImg(name, imgfile):
    #clés Exif
    DATE_KEY = 36867
    MODEL_KEY = 272
    
    img_data[name].update({
        #'size': [getSizeImg(imgfile.size), imgfile.size],
        'size': getSizeImg(imgfile.size),
        'orientation': getOrientationImg(imgfile.size),
        'format': imgfile.format,
    })
    
    getColorsImg(name, imgfile)
    
    exif_data = imgfile._getexif()
    if exif_data is not None:
        if DATE_KEY in exif_data:
            img_data[name]['date'] = exif_data[DATE_KEY]
        if MODEL_KEY in exif_data:
            img_data[name]['model'] = exif_data[MODEL_KEY]

def getOrientationImg(size): #(largeur,hauteur)
    width,height = size
    return "landscape" if width > height else "portrait" if height > width else "squared"
    
def getSizeImg(size): #(largeur,hauteur)
    width,height = size
    return "large" if width*height>1920*1080 else "small" if width*height<640*480 else "medium"
    


## *2. Main Color* 

In [5]:
def getColorsImg(name, imgfile, cluster_nbr=2, n_init=1):
    
    img_data[name]['colors'] = []
    
    # Redimensionner l'image pour accélérer le traitement
    imgfile = imgfile.resize((int(imgfile.size[0]/2), int(imgfile.size[1]/2)))

    # Convertir l'image en un tableau numpy
    img_array = numpy.array(imgfile)
    
    try:
        img_vector = img_array.reshape(-1, 3)

        clusters = MiniBatchKMeans(n_clusters=cluster_nbr, n_init=n_init, random_state=0).fit(img_vector) #Random state pour garder la meme seed ppour toutes les images

        for i, center in enumerate(clusters.cluster_centers_):
            rgb = tuple(map(int, center))
        

            img_data[name]['colors'].append(rgb)
            
    except Exception as e:
        print('Pas de couleur : {}'.format(e))
        #img_data[name]['colors'] = [-1]

        
def RGBtoName(rgb):
    # chercher le nom de couleur correspondant au code RGB
    try:
        color_name = webcolors.rgb_to_name(rgb)
    except ValueError:
        # si le nom exact n'est pas trouvé, trouver le nom le plus proche
        min_colors = {}
        for key, name_ in webcolors.CSS3_HEX_TO_NAMES.items():
            r_c, g_c, b_c = webcolors.hex_to_rgb(key)
            rd = (r_c - rgb[0]) ** 2
            gd = (g_c - rgb[1]) ** 2
            bd = (b_c - rgb[2]) ** 2
            min_colors[(rd + gd + bd)] = name_
        color_name = min_colors[min(min_colors.keys())]
    return color_name

## *3. All Meta* 

In [6]:
def openImgGetMeta(name): #name = 'img.jpg'
    try:
        with Image.open("images/"+name) as imgfile:
            #print(imgfile.size, imgfile.format)
            name = name.split(".")[0]
            getMetaImg(name, imgfile)

        imgfile.close()
        return True
    except FileNotFoundError as e:
        print('Impossible de trouver le fichier {}: {}'.format("images/"+name, e))
        return False
    except Exception as e:
        print('Erreur lors de l\'ouverture de l\'image {}: {}'.format("images/"+name, e))
        removeImage(name)
        return False
        
#Obtenir les metas de toutes nos images    
def openGetMetaAllImg(namefolder='images'):
    for image in os.listdir(namefolder):
        openImgGetMeta(image)

openGetMetaAllImg()

Pas de couleur : cannot reshape array of size 67760 into shape (3)


## *4. Export to JSON* 

In [7]:
#Enregistre les metadonnées des images dans un JSON
def exportToJSON(data,name='img_data.json'):
    with open(name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print("done")
        
exportToJSON(data=img_data)

done


## *5. Open JSON* 

In [8]:

def openJSON(file="img_data.json"):
    # Ouvrir le fichier JSON en mode lecture (fermé automatiquement)
    with open(file, 'r') as f:
        data = json.load(f)
    return data
    

## *6. Add categories* 

In [9]:
# Requête API à Wikidata pour récupérer les catégories de l'image
def wikidataAPI(_id,lang):
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={_id}&props=labels|claims&languages={lang}&format=json"
    #if lang != "":
    #    #categorie
    #    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={_id}&props=labels&languages={lang}&format=json"
    #else:
    #    #image
    #    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={_id}&props=claims&format=json"
    response = requests.get(url)
    return json.loads(response.text)


def findTagsImg(_id, lang='en'): #"Q7704028-Q144"
    
    image_id, cat_id = _id.split("-")
    
    # Requête API à Wikidata pour récupérer les catégories de l'image
    img_data = wikidataAPI(image_id,lang)
    cat_data = wikidataAPI(cat_id,lang)
    
    # Extraire le libellé de la catégorie dans la langue souhaitée
    main_tag = cat_data.get('entities', {}).get(cat_id, {}).get('labels', {}).get(lang, {}).get('value', '')
    
    # Trouver les catégories dans la réponse de la requête API sur l'image
    claims = img_data.get("entities", {}).get(image_id, {}).get("claims", {})
    #print([x['mainsnak']['datavalue']['value'] for prop in ["P910", "P373"] for x in claims.get(prop, [])])
    categories = formatCategory([x['mainsnak']['datavalue']['value'] for prop in ["P910", "P373"] for x in claims.get(prop, [])])
            
    if main_tag not in categories:
        categories.append(main_tag)
    
    return categories


def formatCategory(categories):
    categories_ = []
    
    removeCar = ['(', ')']
    splitCar = [',', ' ']
    
    for category in categories:
        if isinstance(category, str):
            for car in removeCar:
                category = category.replace(car, '')
            for car in splitCar:
                parts = category.split(car)
                if len(parts) > 1:
                    categories_.extend(parts[1:])
                    category = parts[0]
            categories_.append(category.lower())
    
    return categories_


def addTagsJSON(image_id = "Q7704028", categories=[]):
    data = openJSON(file="img_data.json")
    data[image_id]['tags'] = categories


def addAllTagsJSON():
    data = openJSON(file="img_data.json")
    for _id in data:
        data[_id]['tags'] = findTagsImg(_id)
    return data


data = addAllTagsJSON()
exportToJSON(data)


done


# **Data Analyses**

## *1. Class User* 

In [142]:
class User:
    def __init__(self, favorite_pictures=None):
        self._favorite_colors = []
        self._favorite_orientations = []
        self._favorite_sizes = []
        self._favorite_tags = []
        self._favorite_pictures = favorite_pictures
        
        self.init_user()

    def get_favorite_pictures(self):
        return self._favorite_pictures

    def __str__(self):
        return f"User with favorite color {self._favorite_colors}, orientation {self._favorite_orientations}, size {self._favorite_sizes}, tags {self._favorite_tags} and favorite pictures {self._favorite_pictures}"
    
    
    def init_user(self):
        for img in self._favorite_pictures:
            img = img.split(".")[0]
            self._favorite_colors += [data[img]["colors"]]
            #self._favorite_colors.append([c/255 for c in data[img]["colors"][0]] if len(data[img]["colors"])>0 else -1)
            self._favorite_orientations.append(data[img]["orientation"])
            self._favorite_tags += [data[img]["tags"]]
            #self._favorite_tags.append(data[img]["tags"][0])
            self._favorite_sizes.append(data[img]["size"])
            
    def get_favorite_tags(self):
        return self._favorite_tags
    
    def get_favorite_orientations(self):
        return self._favorite_orientations
    
    def get_favorite_colors(self):
        return self._favorite_colors
    
    def get_favorite_sizes(self):
        return self._favorite_sizes


## *2. Creation of random Users* 

In [163]:
number_users = 10
users = []

for i in range(number_users):
    
    number_pictures_liked = random.randint(3, 10)
    
    user = User(favorite_pictures=random.sample(os.listdir("images"), k=number_pictures_liked))
    users.append(user)
    #print(str(i+1)+"- "+str(user.get_favorite_pictures()))
    
    #print(str(i+1)+"- "+str(user._favorite_colors))
    #print(str(i+1)+"- "+str(user._favorite_orientations))


## *3. Analyse Users* 

### *Nos données* 

In [202]:
user = users[0]

colors = user.get_favorite_colors()

couleur1 = [couleur[0] if couleur != [0] else -1 for couleur in colors]
couleur2 = [couleur[1] if len(couleur) > 1 else -1 for couleur in colors]
tags = user.get_favorite_tags()
#print(tags)

#DataFrame Utilisateur
dataUser = pd.DataFrame({'taille': user.get_favorite_sizes(), 'orientation': user.get_favorite_orientations(), 'couleur_1':couleur1, 'couleur_2':couleur2, 'tags':user.get_favorite_tags()})
#'couleur': user.get_favorite_colors(), 'tags': user.get_favorite_tags()

dataUser

,taille,orientation,couleur_1,couleur_2,tags
0,large,landscape,"[197, 168, 121]","[60, 47, 26]",[dog]
1,small,portrait,"[75, 75, 55]","[187, 179, 171]",[dog]
2,small,portrait,"[113, 100, 79]","[207, 195, 167]","[dog, thunder]"
3,small,landscape,-1,-1,"[Nuisance, just, dog]"
4,large,portrait,"[63, 61, 69]","[142, 176, 198]",[flag]
5,small,portrait,"[178, 178, 178]","[80, 80, 80]","[strongheart, dog]"
6,large,landscape,"[81, 57, 53]","[194, 153, 118]","[Elizabeth, Warren's, dog, bailey]"


### *1. Preparation des données* 

In [209]:
dataUserPrep = pd.DataFrame()

# Convertir la taille en une valeur numérique
dataUserPrep['taille'] = dataUser['taille'].map({'small': 0, 'medium': 0.5, 'large':1})

# Convertir l'orientation en une valeur numérique
dataUserPrep['orientation'] = dataUser['orientation'].map({'landscape': 0, 'portrait': 1, 'squared':0.5})

dataUserPrep[['r1','g1','b1']] = dataUser['couleur_1'],[0]dataUser['couleur_1'][1],[0]dataUser['couleur_1'][2]


dataUserPrep

,taille,orientation,r1,g1,b1
0,1.0,0.0,0,0,0
1,0.0,1.0,0,0,0
2,0.0,1.0,0,0,0
3,0.0,0.0,0,0,0
4,1.0,1.0,0,0,0
5,0.0,1.0,0,0,0
6,1.0,0.0,0,0,0


### *2. Apprentissage et Test* 

# **Data Visualization**

## *1. Number of pictures by year* 

In [ ]:
nb_images_year = {}

for image in data:
    date = data[image].get('date', '')[0:5] #Recupere la date de l'image
    nb_images_year[date] = nb_images_year.get(date,0) +1
    
print(nb_images_year)

#Si on veut retirer les images sans date:
nb_images_year.pop('')

noms = list(nb_images_year.keys())
valeurs = list(nb_images_year.values())

plt.bar(noms, valeurs)
plt.title("Nombre d'images par année")
plt.xlabel("Années")
plt.ylabel("Nombre d'images")
plt.show()

## *2. Number* 

# **Recommendation System**


In [ ]:
Préparez vos données en séparant les tags en colonnes binaires (chaque tag a une colonne binaire qui indique si cette image contient le tag ou non).
Transformez les couleurs dominantes en caractéristiques numériques en utilisant, par exemple, la moyenne de chaque canal RGB.
Divisez les données en un ensemble d'apprentissage et un ensemble de test.
Appliquez des algorithmes de classification et de clustering, tels que K-means, SVM ou Random Forest, sur l'ensemble d'apprentissage.
Évaluez la performance des algorithmes en utilisant des mesures telles que l'exactitude, la précision, le rappel et la F1-score.
Utilisez l'algorithme le plus performant pour prédire les préférences des utilisateurs sur l'ensemble de test.